In [1]:
import numpy as np
import pandas as pd
import faiss
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy

In [2]:
scaler = StandardScaler()

Считываем данные

In [19]:
base = pd.read_csv('../dev/data/base.csv')
train = pd.read_csv('../dev/data/train.csv')
test = pd.read_csv('../dev/data/test.csv')

In [4]:
base.shape, test.shape, train.shape

((2918139, 73), (100000, 73), (100000, 74))

In [5]:
train_t = train.drop('Id', axis=1)

base[base['Id'] == train_t['Target'][0]]

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
598613,675816-base,-63.856552,17.163416,-32.26922,-169.7521,198.87073,-77.22896,-347.360606,13.288679,108.23107,...,-107.259094,80.318184,-151.78957,56.448963,66.886116,45.39171,40.096794,116.98201,-415.750254,-18.19598


In [24]:
test.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,-85.302233,7.669724,-43.842474,-146.119797,111.635071,-73.273042,-441.396759,21.594242,123.077725,126.001062,...,-81.129311,36.778882,-152.341249,14.397762,67.788892,23.250779,73.114446,115.196935,-709.457021,-48.416276
std,25.777321,4.956990,39.138775,20.495541,47.751576,28.515740,279.242585,66.478048,6.504931,64.981697,...,30.691590,25.427103,41.246347,98.695231,1.844522,55.403862,62.056224,21.493081,405.665764,41.292843
min,-190.353330,-11.109877,-217.538420,-220.050890,-81.198990,-176.705030,-791.460243,-265.608430,96.218350,-135.667300,...,-215.152280,-66.265970,-337.632870,-157.593541,60.377728,-210.672800,-175.921780,25.271042,-1297.923999,-209.935760
25%,-103.231790,4.292425,-69.680916,-160.210287,79.101920,-92.789803,-740.620025,-21.804730,118.657689,82.311205,...,-101.690472,19.564135,-179.115082,-70.420034,66.546130,-13.863008,31.494840,100.538862,-1074.464888,-76.359190
50%,-85.296745,7.657888,-43.230835,-146.080365,111.959330,-73.579283,-513.922978,22.138012,123.062568,126.083595,...,-81.027250,36.414150,-151.426035,14.768218,67.800445,23.508739,72.152398,115.280990,-807.029697,-48.670001
75%,-67.422104,10.993900,-17.765821,-132.119278,143.903735,-53.861139,-202.572440,66.097697,127.538967,169.680500,...,-60.274745,53.888420,-124.400438,99.803923,69.040832,60.154775,114.317053,129.770215,-358.400478,-20.283335
max,14.427986,27.409784,134.859800,-57.381890,302.065370,50.255325,109.632035,279.468900,151.822560,392.827150,...,41.390480,141.025270,14.602035,185.094333,75.400475,251.288590,305.937530,201.599980,98.737079,111.831955


In [6]:
model_columns = base.columns[1:]
scaler.fit(base[model_columns])

base[model_columns] = scaler.transform(base[model_columns])
test[model_columns] = scaler.transform(test[model_columns])

In [7]:
base

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,0-base,-1.159216,0.620350,-0.513723,1.398558,2.269043,-1.160385,-0.282884,0.373663,-0.435627,...,1.189049,0.190524,0.073589,-0.894789,-2.262493,-0.510438,0.293004,-0.014051,1.781420,-0.312329
1,1-base,2.075715,1.060423,-0.652491,-1.002705,-1.157244,-0.500495,1.130652,-0.871900,-0.623903,...,-1.272135,0.270119,-0.073569,-1.097493,0.225102,0.014491,1.711283,0.059848,1.853726,-0.281052
2,2-base,1.285417,-0.343342,0.397878,0.666228,0.853956,-0.973671,1.637293,0.654489,0.828686,...,0.086191,0.440159,-1.269686,1.143739,-1.220546,0.773022,0.099669,0.048521,-0.713847,0.365623
3,3-base,-0.051852,-0.816047,-1.118829,-1.936660,-0.754014,0.461441,0.033281,-0.840347,-0.259958,...,0.274980,-1.372853,0.184217,-0.523936,-0.871349,-0.797073,1.014714,0.090076,-0.713847,-0.601807
4,4-base,0.541780,-0.314538,2.284450,0.303705,-2.276181,-1.422075,-0.020200,0.386656,-0.238677,...,0.716557,0.808178,-0.110496,0.725544,-0.562342,-0.470720,-0.157210,0.920175,-0.713847,1.152406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918134,4744755-base,-1.561060,-0.675221,2.145022,1.076266,2.124613,0.855791,-0.868394,-1.423783,0.291078,...,0.284290,0.766146,-0.891135,1.099805,-1.248473,1.543466,-2.082561,0.079706,0.475699,-0.718053
2918135,4744759-base,-0.898663,-0.530123,0.579546,-0.608808,0.049398,-0.597886,1.012274,1.821418,0.845243,...,0.284428,-0.514378,1.555805,-0.431911,0.493593,-0.395871,-1.619490,-1.040661,-0.713847,-1.030208
2918136,4744762-base,-0.433776,-2.064035,-0.690969,0.098481,0.823080,1.788798,-1.352459,-2.001122,0.177137,...,0.757178,1.025621,-0.347378,0.953011,0.439465,0.355254,-0.096658,0.659732,-0.713847,1.257774
2918137,4744763-base,-0.024465,0.167939,0.252204,2.024324,-1.490462,-0.858682,-0.517876,-0.163629,0.370246,...,-0.257554,-1.393629,-0.467622,-1.669100,0.455521,-0.412650,0.317753,0.438073,-0.713847,-0.191579


In [8]:
d = len(model_columns)                          # dimension
nb = base.shape[0]                     # database size
nq = test.shape[0]                 # nb of queries
np.random.seed(134)             # make reproducible

xb = np.ascontiguousarray(base[model_columns].values).astype('float32')
xq = np.ascontiguousarray(test[model_columns].values).astype('float32')

nlist = 1000
k = 10
quantizer = faiss.IndexFlatIP(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist)
assert not index.is_trained
index.train(xb)
assert index.is_trained
index.add(xb)
index.nprobe = 30
D, I = index.search(xq, k)

Формируем таблицу с ответом

In [9]:
print(index.nprobe)

30


In [10]:
D

array([[ 8.076222 , 10.031795 , 13.769162 , ..., 19.93933  , 20.306961 ,
        20.335863 ],
       [29.305496 , 29.933315 , 35.113445 , ..., 57.378166 , 59.924168 ,
        59.970375 ],
       [ 2.6804886, 12.730002 , 17.26474  , ..., 40.75234  , 44.22648  ,
        54.7207   ],
       ...,
       [41.655323 , 42.346085 , 43.034027 , ..., 46.756187 , 46.94151  ,
        47.082375 ],
       [ 1.9209011, 61.416504 , 62.49571  , ..., 65.0027   , 65.68736  ,
        65.774445 ],
       [30.70575  , 35.56262  , 36.158344 , ..., 38.66554  , 39.44521  ,
        40.044304 ]], dtype=float32)

In [11]:
I

array([[2192372, 2177660,  342838, ..., 1405698, 2910875, 1719453],
       [ 157966,   11829,  847800, ..., 2561209, 2123768,  138680],
       [ 431806,  148400,  451870, ...,  359945,  208180, 1706372],
       ...,
       [1392746,  349609,  699739, ..., 1566417, 2249505,  859189],
       [ 319526, 2820290, 2253420, ..., 2236716, 1940917,   27373],
       [2888788, 2505458,  330927, ..., 1096879,   83340,  315821]])

In [12]:
#из таблицы I с индексами кандидатов формируем строку в формате ответа
id_base_dict = dict(base['Id'])

predicted_list = []
for candidates in I:
    predicted_list.append(' '.join([id_base_dict[candidate] for candidate in candidates]))
    
#формируем ответ
baseline_answer = test[['Id']]
baseline_answer['Predicted'] = predicted_list



/var/folders/9z/97gxwh_9275807yn3fyb9t6m0000gn/T/ipykernel_37818/3234927958.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_answer['Predicted'] = predicted_list


# Была попытка понять как прикрутить CatBoost для ранжирования ответов - не справился, тк не понял

In [13]:
baseline_answer

,Id,Predicted
0,100000-query,3209652-base 3181043-base 368296-base 645855-b...
1,100001-query,163485-base 11853-base 1000521-base 3890545-ba...
2,100002-query,472256-base 153272-base 496010-base 298322-bas...
3,100003-query,2345993-base 2104072-base 4473809-base 2177262...
4,100004-query,75484-base 1217188-base 722770-base 624427-bas...
...,...,...
99995,199995-query,880713-base 250053-base 2868525-base 2095805-b...
99996,199996-query,1188047-base 2653840-base 991649-base 260043-b...
99997,199997-query,1801591-base 376082-base 804666-base 108637-ba...
99998,199998-query,341779-base 4523822-base 3329598-base 1826342-...


In [14]:
base[base['Id'] == train['Target'][0]][model_columns].values

array([[ 0.8988244 ,  1.83376362,  0.31869163, -1.16489497,  1.88905556,
        -0.18582176,  0.16514791, -0.11000544, -2.43123041, -1.25141232,
         0.89727319,  0.43148261,  0.11251276,  0.15258964,  2.33453339,
        -1.11341349, -0.05690248, -2.46956987, -1.4156506 , -0.08449924,
         0.34713388,  0.62431793, -1.26306614,  0.47428742,  2.43372871,
         0.42159108, -0.43362492, -0.50926807, -2.14320675, -0.15034427,
        -0.57522101, -0.35922692,  0.72183999,  1.52661836, -0.20471277,
        -1.44197761,  1.25668775, -0.34104439,  0.37731333,  0.54270616,
        -0.89426685, -0.18035163, -0.29362582,  0.1608612 ,  0.71248356,
         0.93715529, -0.38853592, -1.24531455, -0.5900144 , -1.95932186,
        -0.62209951,  0.00314888,  0.58806493, -0.40776902, -0.769536  ,
         0.60529242,  0.93395078, -0.22825364, -1.79930073,  1.52159573,
        -0.04502277, -0.59186894, -0.92710299,  1.62780556,  0.0729242 ,
         0.42745987, -0.49664134,  0.39475848, -0.5

In [15]:
ttt = train.iloc[0]

In [16]:
ttt['Target'] = base[base['Id'] == train['Target'][0]][model_columns].values[0]

/var/folders/9z/97gxwh_9275807yn3fyb9t6m0000gn/T/ipykernel_37818/268014651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ttt['Target'] = base[base['Id'] == train['Target'][0]][model_columns].values[0]


In [17]:
ttt

Id                                                  0-query
0                                                -53.882748
1                                                 17.971436
2                                                -42.117104
3                                                -183.93668
                                ...                        
68                                                62.642094
69                                                 134.7636
70                                              -415.750254
71                                               -25.958572
Target    [0.8988243952542447, 1.8337636230589847, 0.318...
Name: 0, Length: 74, dtype: object

In [18]:
#baseline_answer.to_csv('sixth_try.csv', index=False)